In [1]:
#| hide
%load_ext autoreload
%autoreload 2

In [2]:
#| default_exp core

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
from contextlib import contextmanager
from pathlib import Path

from fastcore.basics import patch

In [5]:
#| export
from hwpapi.actions import _Action, _Actions
from hwpapi.dataclasses import CharShape, ParaShape
from hwpapi.functions import (
    check_dll,
    dispatch,
    get_absolute_path,
    get_charshape_pset,
    get_parashape_pset,
    get_rgb_tuple,
    get_value,
    mili2unit,
    point2unit,
    set_charshape_pset,
    set_parashape_pset,
    set_pset,
    unit2mili,
    unit2point,
)

In [6]:
#| export
class App:
    """`App` 클래스는 한컴오피스의 한/글 프로그램과 상호작용하기 위한 인터페이스를 제공합니다.

    메소드
    ----------
    __init__(api=None, is_visible=True) : None
        `App` 클래스의 객체를 초기화합니다. 이 메소드에서는 `api` 객체를 인자로 받습니다.
        만약 `api`가 제공되지 않았을 경우, `wc.gencache.EnsureDispatch("HWPFrame.HwpObject")`를 호출하여
        기본값으로 한/글 프로그램의 COM 객체를 생성합니다. 그리고 `self.api` 속성에 이 객체를 할당합니다.
        `_Actions` 클래스의 객체를 생성하여 `self.actions` 속성에 할당하고, `self.set_visible()` 함수를 호출합니다.

    __str__() : str
        `App` 클래스의 객체를 문자열로 변환하여 반환합니다. 반환되는 문자열은 `"<Hwp App: {self.get_filepath()}>"` 형식을 가집니다.

    __repr__() : str
        `__repr__` 메소드는 `__str__` 메소드와 동일한 기능을 수행합니다.
    """

    def __init__(self, api=None, is_visible=True, dll_path=None):
        """`__init__` 함수에서는 `api` 객체를 인자로 받습니다.
        만약 `api`가 제공되지 않았을 경우, `wc.gencache.EnsureDispatch("HWPFrame.HwpObject")`를 호출하여
        기본값으로 한/글 프로그램의 COM 객체를 생성합니다. 그리고 `self.api` 속성에 이 객체를 할당합니다.
        `_Actions` 클래스의 객체를 생성하여 `self.actions` 속성에 할당하고, `self.set_visible()` 함수를 호출합니다.
        """
        if not api:
            api = dispatch("HWPFrame.HwpObject")
        self.api = api
        self.actions = _Actions(self)
        self.parameters = api.HParameterSet
        self.set_visible(is_visible)
        check_dll(dll_path)
        self.api.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

    def __str__(self):
        return f"<Hwp App: {self.get_filepath()}>"

    __repr__ = __str__

In [7]:
#| export


@patch
def reload(app: App):
    app.api = dispatch("HWPFrame.HwpObject")
    app.set_visible()
    check_dll()
    app.api.RegisterModule("FilePathCheckDLL", "FilePathCheckerModule")

In [8]:
#| export
@patch
def set_visible(app: App, is_visible=True, window_i=0):
    """`set_visible()` 함수는 한/글 프로그램의 창을 화면에 보이거나 숨기기 위해 호출됩니다.
    `is_visible` 인자가 `True`일 경우 창이 화면에 보이고, `False`일 경우 숨깁니다.
    `window_i` 인자는 창의 인덱스를 지정합니다."""

    app.api.XHwpWindows.Item(window_i).Visible = is_visible

In [9]:
#| export
@patch
def get_filepath(app: App):
    """`get_filepath()` 함수는 현재 열려있는 한/글 문서의 경로를 반환합니다."""
    doc = app.api.XHwpDocuments.Active_XHwpDocument
    return doc.FullName

In [10]:
#| export
@patch
def create_action(app: App, action_key: str):
    """
    `create_action` 함수는 `_Action` 클래스의 인스턴스를 생성하고 반환합니다.

    매개변수
    ----------
    app : App
        액션을 생성할 애플리케이션 객체입니다.

    action_key : str
        생성할 액션의 키입니다.

    반환
    ------
    _Action(app, action_key) : _Action object
        입력받은 애플리케이션 객체와 액션 키를 사용하여 생성된 _Action 클래스의 인스턴스를 반환합니다.
    """
    return _Action(app, action_key)

In [11]:
#| export
@patch
def create_parameterset(app: App, action_key: str):
    """
    `create_parameterset` 함수는 특정 액션에 대한 파라미터셋을 생성하고 반환합니다.

    `_action_info` 딕셔너리를 사용하여 주어진 액션 키에 대한 파라미터셋 정보를 조회합니다.
    파라미터셋 정보가 있으면 해당 파라미터셋 객체를 생성하고 반환합니다.

    매개변수
    ----------
    app : App
        파라미터셋을 생성할 애플리케이션 객체입니다.

    action_key : str
        파라미터셋을 생성할 액션의 키입니다.

    반환
    ------
    getattr(app.api.HParameterSet, f"H{pset_key}") : HParameterSet object or None
        해당 액션 키에 대한 파라미터셋 객체를 반환합니다.
        만약 액션 키에 대한 파라미터셋 정보가 없으면 None를 반환합니다.
    """
    pset_key, description = _action_info.get(action_key, None)
    if not pset_key:
        return None
    return getattr(app.api.HParameterSet, f"H{pset_key}")

In [12]:
app = App()

In [13]:
#| export
@patch
def open(app: App, path: str):
    """`open()` 함수는 파일 경로를 인자로 받아 해당 파일을 한/글 프로그램에서 엽니다.
    `get_absolute_path()` 함수를 호출하여 절대 경로로 변환한 후, `api.Open()` 함수를 호출하여 파일을 엽니다.
    열린 파일의 경로를 반환합니다."""
    name = get_absolute_path(path)
    app.api.Open(name)
    return name

In [14]:
app.open("test/공문양식.hwp")

'c:/Users/freed/Documents/python_projects/HwpApi/nbs/02_api/test/공문양식.hwp'

In [15]:
#| export
@patch
def get_hwnd(app:App):
    return app.api.XHwpWindows.Active_XHwpWindow.WindowHandle

In [16]:
app.get_hwnd()

10359136

In [17]:
#| export
@patch
def save(app: App, path=None):
    """
    """

    if not path:
        app.api.Save()
        return app.get_filepath()
    name = get_absolute_path(path)
    extension = Path(name).suffix
    format_ = {
        ".hwp": "HWP",
        ".pdf": "PDF",
        ".hwpx": "HWPML2X",
        ".png": "PNG",
    }.get(extension)

    app.api.SaveAs(name, format_)
    return name

In [18]:
app.save("test.pdf")

'c:/Users/freed/Documents/python_projects/HwpApi/nbs/02_api/test.pdf'

In [19]:
#| export
@patch
def save_block(app: App, path: Path):
    """
    `save_block` 함수는 블록을 저장하고 주소를 반환합니다.

    매개변수
    ----------
    app : App
        액션을 생성할 애플리케이션 객체입니다.

    path : Path
        저장할 주소 입니다.

    반환
    ------
    path : 생성이 성공하면 주소를 반환합니다. 실패하면 None을 반환합니다.
    """

    name = get_absolute_path(path)
    extension = Path(name).suffix
    format_ = {
        ".hwp": "HWP",
        ".pdf": "PDF",
        ".hwpx": "HWPML2X",
        ".png": "PNG",
    }.get(extension)

    action = app.actions.SaveBlockAction()
    p = action.pset
    
    p.filename = name
    p.Format = format_
    action.run()
    return name if Path(name).exists() else None

In [20]:
app.save_block("test2.hwp")

In [21]:
#| export
@patch
def close(app: App):
    """`close()` 함수는 현재 열려있는 문서를 닫습니다."""
    app.api.Run("FileClose")

In [22]:
app.close()

In [23]:
#| export
@patch
def quit(app: App):
    """`quit()` 함수는 한/글 프로그램을 종료합니다."""
    app.api.Run("FileQuit")

In [24]:
#| export
@patch
def get_charshape(app: App):
    """
    `get_charshape` 메소드는 현재 애플리케이션에서 문자 모양을 가져오는 함수입니다.

    매개변수
    ----------
    app : App
        문자 모양을 가져올 애플리케이션 객체입니다.

    반환
    ------
    CharShape
        애플리케이션에서 가져온 문자 모양입니다.
    """
    action = app.actions.CharShape()
    p = action.pset
    return CharShape(p)

In [25]:
app.get_charshape()

<CharShape: hangul_font=함초롬바탕, latin_font=함초롬바탕, text_color=0, fontsize=10.0, bold=0, italic=0, super_script=0, sub_script=0, offset=0, spacing=0, ratio=100, shade_color=#ffffff, shadow_color=#c0c0c0, shadow_offset_x=10, shadow_offset_y=10, strike_out_type=0, strike_out_color=0, underline_type=0, underline_shape=0, underline_color=0, out_line_type=0>

In [26]:
#| export
@patch
def set_charshape(app: App, charshape: CharShape=None, **kwargs):
    """`set_charshape` 함수는 주어진 `CharShape`를 사용하여 애플리케이션의 현재 문단 모양을 설정합니다.

    만약 `charshape`가 `None`인 경우, `CharShape`의 기본 인스턴스를 생성합니다.

    이 함수는 추가로 `kwargs`를 인자로 받아, 이를 `parashape`의 속성에 할당합니다.
    이렇게 수정된 `parashape`는 한/글 문서의 현재 문단 모양을 변경하는데 사용됩니다.

    Parameters
    ----------
    app : App
        `App` 객체입니다. 한/글 애플리케이션에 접근하는데 사용됩니다.
    charshape : CharShape, optional
        한/글 글자 모양을 설정하는데 사용될 `CharShape` 객체입니다. 기본값은 `None`입니다.
    **kwargs :
        `CharShape`의 속성에 할당될 추가 키워드 인자입니다.

    Returns
    -------
    bool
        `set_charshape` 작업의 성공 여부를 나타내는 부울 값입니다.
    """
    if charshape is None:
        charshape = CharShape()

    for key, value in kwargs.items():
        setattr(charshape, key, value)
        
    action = app.actions.CharShape()
    set_pset(action.pset, charshape.todict())
    return action.run()

In [27]:
charshape = CharShape()
charshape.font = "바탕체"
charshape.super_script=1
charshape.sub_script = 1
app.set_charshape(charshape)

True

In [28]:
app.get_charshape()

<CharShape: hangul_font=바탕체, latin_font=바탕체, text_color=0, fontsize=10.0, bold=0, italic=0, super_script=1, sub_script=1, offset=0, spacing=0, ratio=100, shade_color=#ffffff, shadow_color=#c0c0c0, shadow_offset_x=10, shadow_offset_y=10, strike_out_type=0, strike_out_color=0, underline_type=0, underline_shape=0, underline_color=0, out_line_type=0>

In [29]:
charshape = CharShape()
charshape.font = "바탕체"
charshape.super_script=0
app.set_charshape(charshape)
charshape = app.get_charshape()

charshape.super_script == 0

True

In [30]:
#| export
@patch
def get_parashape(app: App):
    """
    `get_parashape` 메소드는 현재 애플리케이션에서 문단 모양을 가져오는 함수입니다.

    매개변수
    ----------
    app : App
        문단 모양을 가져올 애플리케이션 객체입니다.

    반환
    ------
    ParaShape
        애플리케이션에서 가져온 문단 모양입니다.
    """
    action = app.actions.ParagraphShape()
    p = action.pset

    return ParaShape(p)

In [31]:
app.get_parashape()

<ParaShape: left_margin=0, right_margin=0, indentation=0, prev_spacing=0, next_spacing=0, line_spacing_type=Word, line_spacing=160, align_type=Both, break_latin_word=Word, break_non_latin_word=1, snap_to_grid=1, condense=0, widow_orphan=0, keep_with_next=0, page_break_before=0, text_alignment=Font, font_line_height=0, heading_type=None, level=0, border_connect=0, border_text=0, border_offset_left=0, border_offset_right=0, border_offset_top=0, border_offset_bottom=0, tail_type=0, line_wrap=0>

In [32]:
#| export


@patch
def set_parashape(app: App, parashape: ParaShape = None, **kwargs):
    """`set_parashape` 함수는 주어진 `ParaShape`를 사용하여 애플리케이션의 현재 문단 모양을 설정합니다.

    만약 `parashape`가 `None`인 경우, `ParaShape`의 기본 인스턴스를 생성합니다.

    이 함수는 추가로 `kwargs`를 인자로 받아, 이를 `parashape`의 속성에 할당합니다.
    이렇게 수정된 `parashape`는 한/글 문서의 현재 문단 모양을 변경하는데 사용됩니다.

    Parameters
    ----------
    app : App
        `App` 객체입니다. 한/글 애플리케이션에 접근하는데 사용됩니다.
    parashape : ParaShape, optional
        한/글 문단 모양을 설정하는데 사용될 `ParaShape` 객체입니다. 기본값은 `None`입니다.
    **kwargs :
        `ParaShape`의 속성에 할당될 추가 키워드 인자입니다.

    Returns
    -------
    bool
        `set_parashape` 작업의 성공 여부를 나타내는 부울 값입니다.
    """
    if parashape is None:
        parashape = ParaShape()

    for key, value in kwargs.items():
        setattr(parashape, key, value)

    action = app.actions.ParagraphShape()
    set_pset(action.pset, parashape.todict())
    return action.run()

In [33]:
app.set_parashape(line_spacing=100)

True

In [34]:
#| export
@patch
def insert_text(
    app: App,
    text: str,
    charshape: CharShape = None, 
    **kwargs,
):
    """`text를 입력합니다."""
    if not charshape:
        charshape = CharShape()
    for key, value in kwargs.items():
        setattr(charshape, key, value)
    app.set_charshape(charshape)
    
    insert_text = app.actions.InsertText()
    p = insert_text.pset
    p.Text = text
    
    insert_text.run()
    return

In [35]:
app.insert_text("안녕하세요", charshape)

In [36]:
app.set_parashape(ParaShape(right_margin=20, align_type="Left"))

True

In [37]:
parashape = app.actions.ParagraphShape()

In [38]:
parashape.pset.RightMargin

5660

In [39]:
#| export
mask_options = {
    "Normal": 0x00,  # "본문을 대상으로 검색한다.(서브리스트를 검색하지 않는다.)"
    "Char": 0x01,  # "char 타입 컨트롤 마스크를 대상으로 한다.(강제줄나눔, 문단 끝, 하이픈, 묶움빈칸, 고정폭빈칸, 등...)"
    "Inline": 0x02,  # "inline 타입 컨트롤 마스크를 대상으로 한다.(누름틀 필드 끝, 등...)"
    "Ctrl": 0x04,  # "extende 타입 컨트롤 마스크를 대상으로 한다.(바탕쪽, 프레젠테이션, 다단, 누름틀 필드 시작, Shape Object, 머리말, 꼬리말, 각주, 미주, 번호관련 컨트롤, 새 번호 관련 컨트롤, 감추기, 찾아보기, 글자 겹침, 등...)"
    "All": None,
}

scan_spos_keys = {
    "Current": 0x0000,  # "캐럿 위치부터. (시작 위치)",
    "Specified": 0x0010,  # "특정 위치부터. (시작 위치)",
    "Line": 0x0020,  # "줄의 시작부터. (시작 위치)",
    "Paragraph": 0x0030,  # "문단의 시작부터. (시작 위치)"
    "Section": 0x0040,  # "구역의 시작부터. (시작 위치)"
    "List": 0x0050,  # "리스트의 시작부터. (시작 위치)"
    "Control": 0x0060,  # "컨트롤의 시작부터. (시작 위치)"
    "Document": 0x0070,  # "문서의 시작부터. (시작 위치)"
}

scan_epos_keys = {
    "Current": 0x0000,  # "캐럿 위치까지. (끝 위치)"
    "Specified": 0x0001,  # "특정 위치까지. (끝 위치)"
    "Line": 0x0002,  # "줄의 끝까지. (끝 위치)"
    "Paragraph": 0x0003,  # "문단의 끝까지. (끝 위치)"
    "Section": 0x0004,  # "구역의 끝까지. (끝 위치)"
    "List": 0x0005,  # "리스트의 끝까지. (끝 위치)"
    "Control": 0x0006,  # "컨트롤의 끝까지. (끝 위치)"
    "Document": 0x0007,  # "문서의 끝까지. (끝 위치)"
}

scan_directions = {
    "Forward": 0x0000,  # "정뱡향. (검색 방향)"
    "Backward": 0x0100,  # "역방향. (검색 방향)"
}


def _get_text(app):
    """스캔한 텍스트 텍스트 제너레이터"""
    flag, text = 2, ""
    while flag not in [0, 1, 101, 102]:
        flag, text = app.api.GetText()
        yield text


@patch
@contextmanager
def scan(
    app: App,
    option="All",
    selection=False,
    scan_spos="Document",
    scan_epos="Document",
    spara=None,
    spos=None,
    epara=None,
    epos=None,
    scan_direction="Forward",
):
    # set start and end position
    spos_id = get_value(scan_spos_keys, scan_spos)
    epos_id = get_value(scan_epos_keys, scan_epos)
    range_ = spos_id + epos_id
    # if selection
    if selection:
        range_ = 0x00FF  # "검색의 범위를 블록으로 제한."

    # set direction
    direction = get_value(scan_directions, scan_direction)
    range_ = range_ + direction
    app.api.InitScan(
        option=get_value(mask_options, option),
        Range=range_,
        spara=spara,
        spos=spos,
        epara=epara,
        epos=epos,
    )
    yield _get_text(app)
    app.api.ReleaseScan()

In [40]:
#| export


def move_to_line(app: App, text):
    """인자로 전달한 텍스트가 있는 줄의 시작지점으로 이동합니다."""
    with app.scan(scan_spos="Line") as scan:
        for line in scan:
            if text in line:
                return app.move(key="ScanPos")
    return False

In [41]:
#| export
move_ids = {
    "Main": 0,  # 루트 리스트의 특정 위치.(para pos로 위치 지정)
    "CurList": 1,  # 현재 리스트의 특정 위치.(para pos로 위치 지정)
    "TopOfFile": 2,  # 문서의 시작으로 이동.
    "BottomOfFile": 3,  # 문서의 끝으로 이동.
    "TopOfList": 4,  # 현재 리스트의 시작으로 이동
    "BottomOfList": 5,  # 현재 리스트의 끝으로 이동
    "StartOfPara": 6,  # 현재 위치한 문단의 시작으로 이동
    "EndOfPara": 7,  # 현재 위치한 문단의 끝으로 이동
    "StartOfWord": 8,  # 현재 위치한 단어의 시작으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "EndOfWord": 9,  # 현재 위치한 단어의 끝으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextPara": 10,  # 다음 문단의 시작으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevPara": 11,  # 앞 문단의 끝으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextPos": 12,  # 한 글자 뒤로 이동.(서브 리스트를 옮겨 다닐 수 있다.)
    "PrevPos": 13,  # 한 글자 앞으로 이동.(서브 리스트를 옮겨 다닐 수 있다.)
    "NextPosEx": 14,  # 한 글자 뒤로 이동.(서브 리스트를 옮겨 다닐 수 있다. 머리말/꼬리말, 각주/미주, 글상자 포함.)
    "PrevPosEx": 15,  # 한 글자 앞으로 이동.(서브 리스트를 옮겨 다닐 수 있다. 머리말/꼬리말, 각주/미주, 글상자 포함.)
    "NextChar": 16,  # 한 글자 뒤로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevChar": 17,  # 한 글자 앞으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextWord": 18,  # 한 단어 뒤로 이동.(현재 리스트만을 대상으로 동작한다.)
    "PrevWord": 19,  # 한 단어 앞으로 이동.(현재 리스트만을 대상으로 동작한다.)
    "NextLine": 20,  # 한 줄 아래로 이동.
    "PrevLine": 21,  # 한 줄 위로 이동.
    "StartOfLine": 22,  # 현재 위치한 줄의 시작으로 이동.
    "EndOfLine": 23,  # 현재 위치한 줄의 끝으로 이동.
    "ParentList": 24,  # 한 레벨 상위로 이동한다.
    "TopLevelList": 25,  # 탑레벨 리스트로 이동한다.
    "RootList": 26,  # 루트 리스트로 이동한다. 현재 루트 리스트에 위치해 있어 더 이상 상위 리스트가 없을 때는 위치 이동 없이 반환한다. 이동한 후의 위치는 상위 리스트에서 서브리스트가 속한 컨트롤 코드가 위치한 곳이다. 위치 이동시 셀렉션은 무조건 풀린다.
    "CurrentCaret": 27,  # 현재 캐럿이 위치한 곳으로 이동한다. (캐럿 위치가 뷰의 맨 위쪽으로 올라간다. )
    "LeftOfCell": 100,  # 현재 캐럿이 위치한 셀의 왼쪽
    "RightOfCell": 101,  # 현재 캐럿이 위치한 셀의 오른쪽
    "UpOfCell": 102,  # 현재 캐럿이 위치한 셀의 위쪽
    "DownOfCell": 103,  # 현재 캐럿이 위치한 셀의 아래쪽
    "StartOfCell": 104,  # 현재 캐럿이 위치한 셀에서 행(row)의 시작
    "EndOfCell": 105,  # 현재 캐럿이 위치한 셀에서 행(row)의 끝
    "TopOfCell": 106,  # 현재 캐럿이 위치한 셀에서 열(column)의 시작
    "BottomOfCell": 107,  # 현재 캐럿이 위치한 셀에서 열(column)의 끝
    "ScrPos": 200,  # 한/글 문서장에서의 screen 좌표로서 위치를 설정 한다.
    "ScanPos": 201,  # GetText() 실행 후 위치로 이동한다.
}


@patch
def move(app: App, key="ScanPos", para=None, pos=None):
    """키워드를 바탕으로 캐럿 위치를 이동시킵니다."""

    move_id = get_value(move_ids, key)
    return app.api.MovePos(moveID=move_id, Para=para, pos=pos)

In [42]:
get_value(move_ids, "ScanPos")

201

In [43]:
#| export

@patch
def setup_page(
    app: App,  # app
    top=20,  # 윗 여백
    bottom=10,  # 아래 여백
    right=20,  # 오른쪽 여백
    left=20,  # 왼쪽 여백
    header=15,  # 머릿말
    footer=5,  # 꼬릿말
    gutter=0,
):  # 제본
    """
    페이지를 설정합니다.
    """
    action = app.actions.PageSetup()
    p = action.pset

    p.PageDef.TopMargin = app.api.MiliToHwpUnit(top)
    p.PageDef.HeaderLen = app.api.MiliToHwpUnit(header)
    p.PageDef.RightMargin = app.api.MiliToHwpUnit(right)
    p.PageDef.BottomMargin = app.api.MiliToHwpUnit(bottom)
    p.PageDef.FooterLen = app.api.MiliToHwpUnit(footer)
    p.PageDef.LeftMargin = app.api.MiliToHwpUnit(left)
    p.PageDef.GutterLen = app.api.MiliToHwpUnit(gutter)

    return action.run()

In [44]:
#| export
size_options = {
    "realSize": 0,  # 이미지를 원래의 크기로 삽입한다.
    "specificSize": 1,  # width와 height에 지정한 크기로 그림을 삽입한다.
    "cellSize": 2,  # 현재 캐럿이 표의 셀안에 있을 경우, 셀의 크기에 맞게 자동 조정하여 삽입한다.
    "cellSizeWithSameRatio": 3,  # 현재 캐럿이 표의 셀 안에 있을 경우, 셀의 크기에 맞추어 원본 이미지의 가로 세로 비율이 동일하게 확대/축소하여 삽입한다.
}

effects = {
    "RealPicture": 0,  # 원본
    "GrayScale": 1,  # 그레이 스케일
    "BlackWhite": 2,  # 흑백효과
}


@patch
def insert_picture(
    app: App,  # app
    fpath,  # 그림 위치
    width=None,  # 넓이
    height=None,  # 높이
    size_option="realSize",  # 사이즈 옵션
    reverse=False,  # 이미지 반전 여부
    watermark=False,  # 워커마크 여부
    effect="RealPicture",  # 화면 효과
):
    """
    사이즈를 지정하여 사진 삽입
    """
    path = Path(fpath)
    sizeoption = get_value(size_options, size_option)
    effect = get_value(effects, effect)
    return app.api.InsertPicture(
        path.absolute().as_posix(),
        Width=width,
        Height=height,
        sizeoption=sizeoption,
        reverse=reverse,
        watermark=watermark,
        effect=effect,
    )

In [45]:
#| export


@patch
def select_text(app: App, option: str = "Line"):
    """
    한줄을 선택합니다.
    """
    select_options = {
        "Doc": (app.actions.MoveDocBegin, app.actions.MoveSelDocEnd),
        "Para": (app.actions.MoveParaBegin, app.actions.MoveSelParaEnd),
        "Line": (app.actions.MoveLineBegin, app.actions.MoveSelLineEnd),
        "Word": (app.actions.MoveWordBegin, app.actions.MoveSelWordEnd),
    }
    begin, end = select_options.get(option)
    return begin().run(), end().run()

In [46]:
app.select_text()

(True, True)

In [47]:
app.set_charshape(CharShape(font="바탕", fontsize=20))

True

In [48]:
#| export
@patch
def get_selected_text(app: App):
    """
    선택된 영역의 텍스트를 불러온다.
    """
    with app.scan(selection=True) as scan:
        text = "\n".join(scan)
    return text

In [49]:
app.get_selected_text()

'안녕하세요\n'

In [50]:
#| export
@patch
def get_text(app: App, spos="Line", epos="Line"):
    """
    텍스트를 가져옵니다. 기본은 현재 문장입니다.
    "Current":  # "캐럿 위치부터. (현재 위치까지)",
    "Specified": 0x0010,  # "특정 위치부터. (특정 위치까지)",
    "Line": 0x0020,  # "줄의 시작부터. (줄 끝까지)",
    "Paragraph": 0x0030,  # "문단의 시작부터. (문단 끝까지)"
    "Section": 0x0040,  # "구역의 시작부터. (구역의 끝까지)"
    "List": 0x0050,  # "리스트의 시작부터. (리스트 끝까지)"
    "Control": 0x0060,  # "컨트롤의 시작부터. (컨트롤 끝까지)"
    "Document": 0x0070,  # "문서의 시작부터. (문서 끝까지)"
    """
    with app.scan(scan_spos=spos, scan_epos=epos) as txts:
        text = "".join(txts)
    return text

In [51]:
app.get_text()

'안녕하세요\r\n'

In [52]:
#| export
directions = {"Forward": 0, "Backward": 1, "All": 2}

In [53]:
#| export
@patch
def find_text(
    app: App,
    text="",
    charshape: CharShape = None,
    ignore_message=True,  # 메시지 무시 여부
    direction="Forward",  # 찾을 방향
    match_case=False,  # 대소문자 구분
    all_word_forms=False,  # 문자열 결합
    several_words=False,  #  여러 단어 찾기
    use_wild_cards=False,  # 아무개 문자
    whole_word_only=False,  # 온전한 낱말
    replace_mode=False,  # 찾아 바꾸기 모드
    ignore_find_string=False,  # 찾을 문자열 무시
    ignore_replace_string=False,  # 바꿀 문자열 무시
    find_style="",  # 찾을 스타일
    replace_style="",
    find_jaso=False,  # 자소로 찾기
    find_reg_exp=False,  # 정규표현식으로 찾기
    find_type=False,  # 다시 찾기를 할 때 마지막으로 실한 찾기를 할 경우 True, 찾아가기를 할경우 False
):
    """
    text를 찾습니다.
    찾지못하면 False를 반환합니다.
    """

    action = app.actions.RepeatFind()
    p = action.pset

    # set options
    p.FindString = text
    p.IgnoreMessage = ignore_message
    p.MatchCase = match_case
    p.AllWordForms = all_word_forms
    p.Direction = get_value(directions, direction)
    p.SeveralWords = several_words
    p.UseWildCards = use_wild_cards
    p.WholeWordOnly = whole_word_only
    p.ReplaceMode = replace_mode
    p.IgnoreFindString = ignore_find_string
    p.IgnoreReplaceString = ignore_replace_string
    p.FindStyle = find_style
    p.ReplaceStyle = replace_style
    p.FindJaso = find_jaso
    p.FindRegExp = find_reg_exp
    p.FindType = find_type

    # set charshape
    if charshape:
        set_pset(p.FindCharShape, charshape.todict())

    return action.run()

In [54]:
app.find_text("안녕하세요")

False

In [55]:
#| export
@patch
def replace_all(
    app: App,
    old_text="",
    new_text="",
    old_charshape: CharShape = None,
    new_charshape: CharShape = None,
    ignore_message=True,  # 메시지 무시 여부
    direction="All",  # 찾을 방향
    match_case=False,  # 대소문자 구분
    all_word_forms=False,  # 문자열 결합
    several_words=False,  #  여러 단어 찾기
    use_wild_cards=False,  # 아무개 문자
    whole_word_only=False,  # 온전한 낱말
    auto_spell=True,  # 토시 자동 교정
    replace_mode=True,  # 찾아 바꾸기 모드
    ignore_find_string=False,  # 찾을 문자열 무시
    ignore_replace_string=False,  # 바꿀 문자열 무시
    find_style="",  # 찾을 스타일
    replace_style="",  # 바꿀 스타일
    find_jaso=False,  # 자소로 찾기
    find_reg_exp=False,  # 정규표현식으로 찾기
    find_type=True,  # 다시 찾기를 할 때 마지막으로 실한 찾기를 할 경우 True, 찾아가기를 할경우 False
):
    action = app.actions.AllReplace()
    p = action.pset

    # set options
    p.FindString = old_text
    p.ReplaceString = new_text
    p.IgnoreMessage = ignore_message
    p.MatchCase = match_case
    p.AllWordForms = all_word_forms
    p.Direction = get_value(directions, direction)
    p.SeveralWords = several_words
    p.UseWildCards = use_wild_cards
    p.WholeWordOnly = whole_word_only
    p.AutoSpell = auto_spell
    p.ReplaceMode = replace_mode
    p.IgnoreFindString = ignore_find_string
    p.IgnoreReplaceString = ignore_replace_string
    p.FindStyle = find_style
    p.ReplaceStyle = replace_style
    p.FindJaso = find_jaso
    p.FindRegExp = find_reg_exp
    p.FindType = find_type

    # set old charshape

    if old_charshape:
        set_pset(p.FindCharShape, old_charshape.todict())
    if new_charshape:
        set_charshape_pset(p.ReplaceCharShape, new_charshape.todict())

    return action.run()

In [56]:
app.insert_text("t")

In [57]:
app.replace_all("t", "txt", new_charshape=CharShape(hangul_font="가는안상수체", fontsize=21))

True

In [58]:
#| export
@patch
def insert_file(
    app: App,
    fpath,
    keep_charshape=False,
    keep_parashape=False,
    keep_section=False,
    keep_style=False,
):
    """
    파일 끼워 넣기
    """

    action = app.actions.InsertFile()
    p = action.pset
    p.filename = Path(fpath).absolute().as_posix()
    p.KeepCharshape = keep_charshape
    p.KeepParashape = keep_parashape
    p.KeepSection = keep_section
    p.KeepStyle = keep_style

    return action.run()

In [59]:
#| export

thickness_dict = {
    -1: "최소값 (=0.1 mm)",
    0: "0.1 mm",
    1: "0.12 mm",
    2: "0.15 mm",
    3: "0.2 mm",
    4: "0.25 mm",
    5: "0.3 mm",
    6: "0.4 mm",
    7: "0.5 mm",
    8: "0.6 mm",
    9: "0.7 mm",
    10: "1.0 mm",
    11: "1.5 mm",
    12: "2.0 mm",
    13: "3.0 mm",
    14: "4.0 mm",
    15: "5.0 mm",
    16: "최대값 (=5.0 mm)"
}


@patch
def set_cell_border(
    app: App,
    top=None,
    right=None,
    left=None,
    bottom=None,
    horizontal=None,
    vertical=None,
    top_width=None,
    right_width=None,
    left_width=None,
    bottom_width=None,
    horizontal_width=None,
    vertical_width=None,
    top_color=None,
    bottom_color=None,
    left_color=None,
    right_color=None,
    horizontal_color=None,
    vertical_color=None,
):
    attrs = {
        "BorderTypeTop": top,
        "BorderTypeRight": right,
        "BorderTypeLeft": left,
        "BorderTypeBottom": bottom,
        "TypeHorz": horizontal,
        "TypeVert": vertical,
        "BorderWidthTop": get_value(thickness_dict, top_width),
        "BorderWidthRight": get_value(thickness_dict, right_width),
        "BorderWidthLeft": get_value(thickness_dict, left_width),
        "BorderWidthBottom": get_value(thickness_dict, bottom_width),
        "WidthHorz": get_value(thickness_dict, horizontal_width),
        "WidthVert": get_value(thickness_dict, vertical_width),
        "BorderColorTop": app.api.RGBColor(get_rgb_tuple(top_color))
        if top_color
        else None,
        "BorderColorRight": app.api.RGBColor(get_rgb_tuple(right_color))
        if right_color
        else None,
        "BorderColorLeft": app.api.RGBColor(get_rgb_tuple(left_color))
        if left_color
        else None,
        "BorderColorBottom": app.api.RGBColor(get_rgb_tuple(bottom_color))
        if bottom_color
        else None,
        "ColorHorz": app.api.RGBColor(get_rgb_tuple(horizontal_color))
        if horizontal_color
        else None,
        "ColorVert": app.api.RGBColor(get_rgb_tuple(vertical_color))
        if vertical_color
        else None,
    }

    action = app.actions.CellFill()
    p = action.pset

    for key, value in attrs.items():
        if value is None:
            continue
        setattr(p, key, value)

    return action.run()

In [60]:
#| export
@patch
def set_cell_color(
    app: App, bg_color=None, hatch_color="#000000", hatch_style=6, alpha=None
):
    fill_type = windows_brush = None
    if bg_color:
        fill_type = 1
        windows_brush = 1

    attrs = {
        "type": fill_type,
        "WindowsBrush": windows_brush,
        "WinBrushFaceColor": app.api.RGBColor(*get_rgb_tuple(bg_color))
        if bg_color
        else None,
        "WinBrushAlpha": alpha,
        "WinBrushHatchColor": app.api.RGBColor(*get_rgb_tuple(hatch_color))
        if hatch_color
        else None,
        "WinBrushFaceStyle": hatch_style,
    }

    action = app.actions.CellBorderFill()
    p = action.pset

    for key, value in attrs.items():
        if value == None:
            continue
        setattr(p.FillAttr, key, value)

    return action.run()

In [61]:
app = App()

In [62]:
app.set_charshape(CharShape(hangul_font="문화바탕", font_type=2))

True

In [63]:
action = app.actions.CharShape()

In [64]:
app.get_charshape()

<CharShape: hangul_font=문화바탕, latin_font=함초롬바탕, text_color=0, fontsize=10.0, bold=0, italic=0, super_script=0, sub_script=0, offset=0, spacing=0, ratio=100, shade_color=#ffffff, shadow_color=#c0c0c0, shadow_offset_x=10, shadow_offset_y=10, strike_out_type=0, strike_out_color=0, underline_type=0, underline_shape=0, underline_color=0, out_line_type=0>

In [65]:
app.api.HwpLineWidth("1.0mm")

10

In [66]:
app.set_cell_border(bottom=3)

False

In [117]:
app.set_cell_color(bg_color="#123456")

True

In [69]:
# app.api.CurSelectedCtrl.CtrlID

In [70]:
# 동작은 되나 너무 느리다. 
# 빠르게 할 수 있는 방법을 찾아보자. 
# 양식을 설정할 수 있는 방법을 생각해 보자.
# 어떻게 해야 위아래로 값이 같은 셀을 병합할 수 있을지 생각해 보자.
# block_input을 해야 그나마 쓸 수 있을 듯

def insert_dataframe(app, df):
    action = app.actions.TableCreate()
    p = action.pset
    p.WidthType = 1    # fit size
    p.Rows = df.shape[0]+1  # add header
    p.Cols = df.shape[1]
    action.run()

    for col in df.columns:
        app.insert_text(col)
        app.move("RightOfCell")
    for i, row in df.iterrows():
        for item in row.values:
            app.insert_text(item)
            app.move("RightOfCell")
    return app.api.ParentCtrl

In [71]:
## 더 빠르고 안정적일 듯
# 초단위로 걸려서 block_input을 해야 할 듯 
# value 중에서 tab이 있으면 replace는 해야 할 듯

def convert_to_text(value):
    value = str(value).replace("\t", "    ")
    return value

def insert_dataframe(app:App, df):
    start_pos = app.api.GetPos()
    text = ""
    text += "\t".join(map(convert_to_text, df.columns))
    for i, row in df.iterrows():
        text += "\r\n"
        text += "\t".join(map(convert_to_text, row.values))
    app.insert_text(text)
    end_pos = app.api.GetPos()

    app.api.SelectText(start_pos[1], start_pos[2], end_pos[1], end_pos[2])

    return app.actions.TableStringToTable().run()

In [72]:
def get_pagewidth(app:App):
    action = app.actions.PageSetup()
    p = action.pset.PageDef
    return unit2mili(p.PaperWidth - (p.LeftMargin + p.RightMargin))

In [73]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.random(100).reshape(10, 10))

In [92]:
dir(app.api)

['Application',
 'ArcType',
 'AutoNumType',
 'BorderShape',
 'BreakWordLatin',
 'BrushType',
 'CLSID',
 'Canonical',
 'CellApply',
 'CellShape',
 'CharShadowType',
 'CharShape',
 'CheckXObject',
 'Clear',
 'ColDefType',
 'ColLayoutType',
 'ConvertPUAHangulToUnicode',
 'CreateAction',
 'CreateField',
 'CreateID',
 'CreateMode',
 'CreatePageImage',
 'CreateSet',
 'CrookedSlash',
 'CurFieldState',
 'CurMetatagState',
 'CurSelectedCtrl',
 'DSMark',
 'DbfCodeType',
 'DeleteCtrl',
 'Delimiter',
 'DrawAspect',
 'DrawFillImage',
 'DrawShadowType',
 'EditMode',
 'Encrypt',
 'EndSize',
 'EndStyle',
 'EngineProperties',
 'ExportStyle',
 'FieldExist',
 'FileTranslate',
 'FillAreaType',
 'FindCtrl',
 'FindDir',
 'FindPrivateInfo',
 'FontType',
 'GetBinDataPath',
 'GetCurFieldName',
 'GetCurMetatagName',
 'GetFieldList',
 'GetFieldText',
 'GetFileInfo',
 'GetFontList',
 'GetHeadingString',
 'GetMessageBoxMode',
 'GetMetatagList',
 'GetMetatagNameText',
 'GetMousePos',
 'GetPageText',
 'GetPos',
 'Ge

## 

cell을 순환하면서 모양을 잡을 수 있을 것으로 예상된다.
6개 값을 받아서 정렬하면 되지 않을까 싶다.


In [113]:
insert_dataframe(app, df)
app.api.FindCtrl()
app.actions.ShapeObjTextBoxEdit().run()
app.set_cell_border(False, False)
for _ in range(10):
    app.actions.TableCellBlockRow().run()
    app.set_cell_color(bg_color=(253, 123, 253), hatch_style=6)
    app.set_cell_border(False, False, False, False)

    app.actions.TableCellBlock().run()
    app.move("DownOfCell")

True

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()